In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}','volbuy':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}','sell_price':'{:.2f}','diff':'{:.2f}','cost':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}',
    'amount':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
    'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','pct':'{:,.2f}%'
}
cols = 'trade name qty price reason market active'.split()

today = date.today()
print(today)

2025-09-07


In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.strftime("%Y-%m-%d")
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-09-07
yesterday: 2025-09-05 00:00:00


In [3]:
# find the beginning of the week for the given yesterday
week_start = yesterday.to_period('W').start_time
week_end = yesterday.to_period('W').end_time
week_start = week_start.date()
week_end = week_end.date()
print(f'week start: {week_start}')
print(f'week end: {week_end}')

week start: 2025-09-01
week end: 2025-09-07


### Restart and Run All Cells

In [5]:
sql = '''
    SELECT name, buys.date AS buy_date, sells.date AS sell_date,
    sells.price AS sell_price, buys.price AS buy_price, 
    (sells.price - buys.price) AS diff, qty, 
    (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
    (sells.price - buys.price) * qty AS gross, 
    ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
    FROM sells JOIN buys ON sells.buy_id = buys.id
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE sells.date BETWEEN '{}' AND '{}'
    ORDER BY name, sells.date
'''.format(week_start, week_end)
print(sql)


    SELECT name, buys.date AS buy_date, sells.date AS sell_date,
    sells.price AS sell_price, buys.price AS buy_price, 
    (sells.price - buys.price) AS diff, qty, 
    (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
    (sells.price - buys.price) * qty AS gross, 
    ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
    FROM sells JOIN buys ON sells.buy_id = buys.id
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE sells.date BETWEEN '2025-09-01' AND '2025-09-07'
    ORDER BY name, sells.date



In [6]:
pf_sells = pd.read_sql(sql, conpf)
pf_sells.style.format(format_dict)

,name,buy_date,sell_date,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit
0,DIF,2025-04-09,2025-09-03,8.30,7.50,0.80,"5,000","41,500.00","37,500.00","4,000.00",10.67%,"3,825.02"


In [7]:
sells_grp = pf_sells.groupby(['name'])[['qty','sell_amt','buy_amt','gross','profit']].sum()
#sells_grp.style.format(format_dict)
sells_grp

,qty,sell_amt,buy_amt,gross,profit
name,,,,,
DIF,5000,41500.0,37500.0,4000.0,3825.02


In [8]:
pf_sells[['sell_amt','buy_amt','gross','profit']].sum()

sell_amt    41500.00
buy_amt     37500.00
gross        4000.00
profit       3825.02
dtype: float64

In [9]:
sql = '''
    SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
    qty, (buys.price * qty) AS buy_amt 
    FROM buys
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE buys.date BETWEEN '{}' AND '{}'
    ORDER BY name, buys.date
'''.format(week_start, week_end)
print(sql)


    SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
    qty, (buys.price * qty) AS buy_amt 
    FROM buys
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE buys.date BETWEEN '2025-09-01' AND '2025-09-07'
    ORDER BY name, buys.date



In [10]:
pf_buys = pd.read_sql(sql, conpf)
pf_buys.style.format(format_dict)

,name,buy_date,buy_price,qty,buy_amt
0,PTT,2025-09-01,31.00,"2,500","77,500.00"


In [11]:
pf_buys[['buy_amt']].sum()

buy_amt    77500.0
dtype: float64

In [12]:
pf_sells[['buy_amt']].sum()

buy_amt    37500.0
dtype: float64

In [13]:
diff_amt = pf_buys[['buy_amt']].sum() - pf_sells[['buy_amt']].sum()
diff_amt

buy_amt    40000.0
dtype: float64

In [14]:
sql = '''
SELECT name, available_qty, cost, status
FROM stocks
'''
lite_stocks = pd.read_sql(sql, conlite)
lite_stocks.set_index('name', inplace=True)
lite_stocks.style.format(format_dict)

,available_qty,cost,status
name,,,
AH,"1,200",37.00,X
AIMIRT,"12,500",10.90,B
AWC,"9,000",4.96,X
BCH,"4,000",21.70,X
CPNREIT,"55,000",18.00,X
DIF,"50,000",12.17,S
GVREIT,"75,000",7.64,B
IVL,"7,200",40.00,B
3BBIF,"120,000",10.10,X


In [15]:
df_merge = pd.merge(sells_grp,lite_stocks,on='name',how='inner')
df_merge.style.format(format_dict)

,qty,sell_amt,buy_amt,gross,profit,available_qty,cost,status
name,,,,,,,,
DIF,"5,000","41,500.00","37,500.00","4,000.00","3,825.02","50,000",12.17,S


In [16]:
sql = '''
SELECT name, SUM(volbuy) AS volbuy, price
FROM buy
WHERE active = 1
GROUP BY name
ORDER BY name'''
df_buy = pd.read_sql(sql, const)
df_buy.volbuy = df_buy.volbuy.astype(int)
df_buy.style.format(format_dict)

,name,volbuy,price
0,3BBIF,"120,000",10.10
1,AH,"1,200",37.00
2,AIMIRT,"17,500",10.60
3,AWC,"9,000",4.96
4,BCH,"4,000",21.70
5,CPF,"5,000",24.10
6,CPNREIT,"55,000",18.00
7,DIF,"45,000",12.70
8,GVREIT,"69,000",7.75
9,IVL,"7,200",40.00


In [17]:
df_merge2 = pd.merge(df_buy,lite_stocks,on='name',how='inner')
df_merge2.style.format(format_dict)

,name,volbuy,price,available_qty,cost,status
0,3BBIF,"120,000",10.10,"120,000",10.10,X
1,AH,"1,200",37.00,"1,200",37.00,X
2,AIMIRT,"17,500",10.60,"12,500",10.90,B
3,AWC,"9,000",4.96,"9,000",4.96,X
4,BCH,"4,000",21.70,"4,000",21.70,X
5,CPF,"5,000",24.10,0,0.00,B
6,CPNREIT,"55,000",18.00,"55,000",18.00,X
7,DIF,"45,000",12.70,"50,000",12.17,S
8,GVREIT,"69,000",7.75,"75,000",7.64,B
9,IVL,"7,200",40.00,"7,200",40.00,B


In [18]:
mask1 = df_merge2.volbuy != df_merge2.available_qty
mask2 = df_merge2.price != df_merge2.cost
df_merge2[mask1 | mask2].style.format(format_dict)

,name,volbuy,price,available_qty,cost,status
2,AIMIRT,"17,500",10.60,"12,500",10.90,B
5,CPF,"5,000",24.10,0,0.00,B
7,DIF,"45,000",12.70,"50,000",12.17,S
8,GVREIT,"69,000",7.75,"75,000",7.64,B
12,MCS,"75,000",15.31,"81,000",14.80,S
14,ORI,"50,000",9.00,"50,000",2.20,B
16,PTT,"5,000",31.50,"2,500",32.00,S
18,RCL,"27,000",38.80,"27,000",38.75,B
23,SYNEX,"17,500",26.40,"15,000",28.70,S
24,TFFIF,"20,000",7.20,"20,000",7.50,B


### New process

In [20]:
sql = '''
SELECT trade, O.name, O.qty, price, S.qty, buy_target
FROM orders O
JOIN stocks S
ON O.name = S.name
WHERE trade = "B" AND price <> buy_target
ORDER BY trade, O.name
'''
lite_orders = pd.read_sql(sql, conlite)
lite_orders.style.format(format_dict)

,trade,name,qty,price,qty,buy_target
0,B,GVREIT,"5,000",6.20,"6,000",6.00
1,B,WHAIR,"10,000",4.30,"10,000",4.90


In [21]:
sql = '''
SELECT trade, O.name, O.qty, price, S.qty, sell_target
FROM orders O
JOIN stocks S
ON O.name = S.name
WHERE trade = "S" AND price <> sell_target
'''
lite_orders = pd.read_sql(sql, conlite)
lite_orders.style.format(format_dict)

,trade,name,qty,price,qty,sell_target
0,S,DIF,"5,000",8.30,"10,000",9.30


In [22]:
const.close()
conmy.close()
conlite.close()
conpf.close()

In [23]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-09-07 22:02:58
